# Capacity Constraints Analysis Example

This notebook demonstrates how to perform capacity constraints analysis for health interventions.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Import the capacity constraints module
from capacity_constraints_engine import analyze_capacity_constraints, analyze_implementation_costs, QueueingModel

In [ ]:
# Define treatment demand and capacity parameters
treatment_demand = pd.DataFrame({
    'strategy': ['ECT', 'IV-KA', 'PO-KA', 'ECT', 'IV-KA', 'PO-KA'],
    'demand': [100, 50, 75, 120, 60, 90],  # Patients per month
    'period': ['current', 'current', 'current', 'projected', 'projected', 'projected']
})

resource_capacity = {
    'treatment_slots': 80,  # Monthly treatment capacity
    'staff_hours_per_month': 1600,
    'equipment_units': 10
}

# Run capacity constraints analysis
capacity_results = analyze_capacity_constraints(treatment_demand, resource_capacity, time_horizon=365)

# Print results
print("Capacity Constraints Analysis Results")
print("="*50)
print("\nWaiting Times:")
print(capacity_results.waiting_times.head(10))

print("\nUtilization Rates:")
print(capacity_results.utilization_rates.head(10))

print("\nBottleneck Analysis:")
print(capacity_results.bottleneck_analysis)

In [ ]:
# Visualize capacity constraints results
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. Utilization rates by strategy and capacity multiplier
util_pivot = capacity_results.utilization_rates.pivot(index='strategy', columns='capacity_multiplier', values='utilization_rate')
sns.heatmap(util_pivot, annot=True, fmt='.2f', cbar_kws={'label': 'Utilization Rate'}, ax=ax[0,0])
ax[0,0].set_title('System Utilization Rate by Strategy and Capacity')

# 2. Mean waiting times
wait_pivot = capacity_results.waiting_times.pivot(index='strategy', columns='capacity_multiplier', values='mean_waiting_days')
sns.heatmap(wait_pivot, annot=True, fmt='.1f', cbar_kws={'label': 'Waiting Days'}, ax=ax[0,1])
ax[0,1].set_title('Mean Waiting Time by Strategy and Capacity')

# 3. Bottleneck indicators
bottleneck_data = capacity_results.bottleneck_analysis
bottleneck_map = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
bottleneck_numeric = [bottleneck_map[val] for val in bottleneck_data['bottleneck_indicator']]
ax[1,0].bar(range(len(bottleneck_data)), bottleneck_numeric, tick_label=bottleneck_data['strategy'])
ax[1,0].set_ylabel('Bottleneck Level (1=Low, 4=Critical)')
ax[1,0].set_title('Bottleneck Analysis by Strategy')
ax[1,0].set_yticks([1, 2, 3, 4])
ax[1,0].set_yticklabels(['Low', 'Medium', 'High', 'Critical'])

# 4. Capacity requirements
if not capacity_results.capacity_requirements.empty:
    ax[1,1].bar(capacity_results.capacity_requirements['strategy'], 
                capacity_results.capacity_requirements['capacity_multiplier'], 
                color='green', alpha=0.7)
    ax[1,1].axhline(y=1.0, color='red', linestyle='--', label='Current Capacity', alpha=0.7)
    ax[1,1].set_ylabel('Capacity Multiplier Needed')
    ax[1,1].set_title('Minimum Capacity Multiplier for Acceptable Waiting Times')
    ax[1,1].legend()
else:
    ax[1,1].text(0.5, 0.5, 'No capacity requirement data', 
                 horizontalalignment='center', verticalalignment='center',
                 transform=ax[1,1].transAxes)
    ax[1,1].set_title('Capacity Requirements')

plt.tight_layout()
plt.show()

In [ ]:
# Run implementation costs analysis
treatment_volumes = pd.DataFrame({
    'strategy': ['ECT', 'IV-KA', 'PO-KA'],
    'annual_volume': [1200, 600, 900]  # Annual patient volume
})

# Default cost parameters
cost_parameters = {}

# Run implementation costs analysis
impl_results = analyze_implementation_costs(treatment_volumes, cost_parameters, time_horizon=5)

print("Implementation Costs Analysis Results")
print("="*50)
print("\nStartup Costs:")
print(impl_results.startup_costs)

print("\nOperational Costs:")
print(impl_results.operational_costs)

print("\nTraining Costs:")
print(impl_results.training_costs)

print("\nTotal Costs:")
print(impl_results.total_costs)

print("\nBreakeven Analysis:")
print(impl_results.breakeven_analysis)

In [ ]:
# Visualize implementation costs results
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. Startup costs
ax[0,0].bar(impl_results.startup_costs['strategy'], impl_results.startup_costs['total_startup'], 
            color='steelblue', alpha=0.7)
ax[0,0].set_ylabel('Cost ($AUD)')
ax[0,0].set_title('Startup Implementation Costs by Strategy')
ax[0,0].grid(True, alpha=0.3)

# 2. Annual operational costs
ax[0,1].bar(impl_results.operational_costs['strategy'], impl_results.operational_costs['total_operational'], 
            color='lightgreen', alpha=0.7)
ax[0,1].set_ylabel('Annual Cost ($AUD)')
ax[0,1].set_title('Annual Operational Costs by Strategy')
ax[0,1].grid(True, alpha=0.3)

# 3. Total 5-year costs
ax[1,0].bar(impl_results.total_costs['strategy'], impl_results.total_costs['total_5year_cost'], 
            color='orange', alpha=0.7)
ax[1,0].set_ylabel('Total Cost ($AUD)')
ax[1,0].set_title('Total 5-Year Implementation Costs by Strategy')
ax[1,0].grid(True, alpha=0.3)

# 4. Breakeven analysis
breakeven_numeric = []
for val in impl_results.breakeven_analysis['breakeven_year']:
    try:
        breakeven_numeric.append(int(val))
    except ValueError:
        breakeven_numeric.append(6)  # Use 6 to indicate >5 years
        
colors = ['red' if val > 5 else 'green' for val in breakeven_numeric]
bars = ax[1,1].bar(range(len(breakeven_numeric)), breakeven_numeric, color=colors, alpha=0.7)
ax[1,1].set_xticks(range(len(impl_results.breakeven_analysis['strategy'])))
ax[1,1].set_xticklabels(impl_results.breakeven_analysis['strategy'])
ax[1,1].set_ylabel('Breakeven Year (or >5)')
ax[1,1].set_title('Breakeven Analysis (Green=Breakeven within 5 years, Red=Not within 5 years)')
ax[1,1].grid(True, alpha=0.3)
ax[1,1].set_ylim(0, 7)

plt.tight_layout()
plt.show()

In [ ]:
# Create a combined capacity and cost analysis
print("Combined Capacity and Cost Analysis")
print("="*60)

# Merge capacity and cost data
capacity_summary = capacity_results.bottleneck_analysis[['strategy', 'utilization_rate', 'bottleneck_indicator']].copy()
cost_summary = impl_results.total_costs[['strategy', 'total_5year_cost']].copy()

# Merge the data
combined_analysis = pd.merge(capacity_summary, cost_summary, on='strategy')

print(combined_analysis)

# Calculate a capacity-cost score (lower is better)
# Normalize values to 0-1 scale
max_cost = combined_analysis['total_5year_cost'].max()
min_cost = combined_analysis['total_5year_cost'].min()
combined_analysis['cost_score'] = (combined_analysis['total_5year_cost'] - min_cost) / (max_cost - min_cost)

# Map bottleneck indicator to numeric
bottleneck_map = {'Low': 0, 'Medium': 0.3, 'High': 0.7, 'Critical': 1.0}
combined_analysis['bottleneck_score'] = combined_analysis['bottleneck_indicator'].map(bottleneck_map)

# Calculate composite score (50% cost, 50% capacity)
combined_analysis['composite_score'] = (combined_analysis['cost_score'] + combined_analysis['bottleneck_score']) / 2
combined_analysis['composite_rank'] = combined_analysis['composite_score'].rank()

print(f"\n\nStrategy Ranking (Composite Score - Lower is Better):")
for _, row in combined_analysis.sort_values('composite_score').iterrows():
    print(f"  {int(row['composite_rank'])}. {row['strategy']} - Score: {row['composite_score']:.3f} "
          f"(Cost: {row['cost_score']:.3f}, Capacity: {row['bottleneck_score']:.3f})")

In [ ]:
# Visualize the combined analysis
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Cost vs Capacity Utilization scatter plot
colors = ['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in combined_analysis['strategy']]
scatter = ax[0].scatter(combined_analysis['utilization_rate'], 
                        combined_analysis['total_5year_cost'],
                        s=100, c=colors, alpha=0.7)

for i, row in combined_analysis.iterrows():
    ax[0].annotate(row['strategy'], 
                   (row['utilization_rate'], row['total_5year_cost']),
                   textcoords="offset points", xytext=(5,5), ha='left')

ax[0].set_xlabel('System Utilization Rate')
ax[0].set_ylabel('Total 5-Year Implementation Cost ($AUD)')
ax[0].set_title('Cost vs Capacity Utilization by Strategy')
ax[0].grid(True, alpha=0.3)

# Composite score bar chart
bottleneck_numeric = combined_analysis['bottleneck_score']
cost_numeric = combined_analysis['cost_score']
x = np.arange(len(combined_analysis))
width = 0.35

ax[1].bar(x - width/2, cost_numeric, width, label='Cost Score', alpha=0.7)
ax[1].bar(x + width/2, bottleneck_numeric, width, label='Capacity Score', alpha=0.7)
ax[1].set_xlabel('Strategy')
ax[1].set_ylabel('Normalized Score (0-1)')
ax[1].set_title('Cost vs Capacity Constraints Scores')
ax[1].set_xticks(x)
ax[1].set_xticklabels(combined_analysis['strategy'])
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Next Steps

1. Integrate with actual treatment demand and capacity data
2. Consider geographic distribution of capacity
3. Model capacity expansion over time
4. Evaluate impact on patient outcomes